In [1]:
import numpy as np

### First task
*Matrix:*
$$\begin{pmatrix} 7.35272 & 0.88255 & -2.270052 \\ 0.88255 & 5.58351 & 0.528167 \\ -2.27005 & 0.528167 & 4.430329 \end{pmatrix} = \begin{pmatrix} 1 \\ 0 \\ 0 \end{pmatrix}$$ 

In [2]:
A = np.array([[7.35272, 0.88255, - 2.270052],
               [0.88255, 5.58351, 0.528167],
               [- 2.27005, 0.528167, 4.430329]])
B = np.array([[1], [0], [0]])
matrix = np.array([[7.35272, 0.88255, - 2.270052, 1],
               [0.88255, 5.58351, 0.528167, 0],
               [- 2.27005, 0.528167, 4.430329, 0]])

LU разложение


In [3]:
def decompose_to_LU(A):
    """Decompose matrix of coefficients to L and U matrices.
     L and U triangular matrices will be represented in a single nxn matrix.
    :param a: numpy matrix of coefficients
    :return: numpy LU matrix
    """
    # create emtpy LU-matrix
    lu_matrix = np.matrix(np.zeros([A.shape[0], A.shape[1]]))
    n = A.shape[0]

    for k in range(n):
        # calculate all residual k-row elements
        for j in range(k, n):
            lu_matrix[k, j] = A[k, j] - lu_matrix[k, :k] * lu_matrix[:k, j]
        # calculate all residual k-column elemetns
        for i in range(k + 1, n):
            lu_matrix[i, k] = (A[i, k] - lu_matrix[i, : k] * lu_matrix[: k, k]) / lu_matrix[k, k]

    return lu_matrix

def get_L(m):
    """Get triangular L matrix from a single LU-matrix
    :param m: numpy LU-matrix
    :return: numpy triangular L matrix
    """
    L = m.copy()
    for i in range(L.shape[0]):
            L[i, i] = 1
            L[i, i+1 :] = 0
    return np.matrix(L)


def get_U(m):
    """Get triangular U matrix from a single LU-matrix
    :param m: numpy LU-matrix
    :return: numpy triangular U matrix
    """
    U = m.copy()
    for i in range(1, U.shape[0]):
        U[i, :i] = 0
    return U

get_U(A)

array([[ 7.35272 ,  0.88255 , -2.270052],
       [ 0.      ,  5.58351 ,  0.528167],
       [ 0.      ,  0.      ,  4.430329]])

In [10]:
def solve_LU(lu_matrix, b):
    """Solve system of equations from given LU-matrix and vector b of absolute terms.
    :param lu_matrix: numpy LU-matrix
    :param b: numpy matrix of absolute terms [n x 1]
    :return: numpy matrix of answers [n x 1]
    """
    # get supporting vector y
    y = np.matrix(np.zeros([lu_matrix.shape[0], 1]))
    for i in range(y.shape[0]):
        y[i, 0] = b[i, 0] - lu_matrix[i, :i] * y[:i]

    # get vector of answers x
    x = np.matrix(np.zeros([lu_matrix.shape[0], 1]))
    for i in range(1, x.shape[0] + 1):
        x[-i, 0] = (y[-i] - lu_matrix[-i, -i:] * x[-i:, 0] )/ lu_matrix[-i, -i]

    return x

Обратная матрица через LU разложение

In [12]:
def reverse_LU(A):
    identy = np.identity(3)
    inverse = np.matrix(np.zeros((3, 3)))
    for i in range(inverse.shape[0]): 
        B = np.matrix(np.zeros((3, 1)))
        B[i] = 1
        x = solve_LU(decompose_to_LU(A), B)
        for k in range(inverse.shape[0]): 
            inverse[k, i] = x[k, 0] 
    return inverse

reverse_LU(A)

matrix([[ 0.16810435, -0.03511503,  0.0903211 ],
        [-0.03511502,  0.18847669, -0.04046203],
        [ 0.09032103, -0.04046202,  0.2768201 ]])

Проверка результата

In [6]:
from numpy import linalg as LA
a_inv = LA.inv(A)
a_inv

array([[ 0.16810435, -0.03511503,  0.0903211 ],
       [-0.03511502,  0.18847669, -0.04046203],
       [ 0.09032103, -0.04046202,  0.2768201 ]])

Число обусловленности

In [13]:
m = LA.norm(A) * LA.norm(reverse_LU(A))
m

4.365265725678259

Метод Гаусса с выбором ведущего элемента

In [8]:
def bubble_max_row(m, col):
    """Replace m[col] row with the one of the underlying rows with the modulo greatest first element.
    :param m: matrix (list of lists)
    :param col: index of the column/row from which underlying search will be launched
    :return: None. Function changes the matrix structure.
    """
    max_element = m[col][col]
    max_row = col
    for i in range(col + 1, len(m)):
        if abs(m[i][col]) > abs(max_element):
            max_element = m[i][col]
            max_row = i
    if max_row != col:
        m[col], m[max_row] = m[max_row], m[col]


def solve_gauss(m):
    """Solve linear equations system with gaussian method.
    :param m: matrix (list of lists)
    :return: None
    """
    n = len(m)
    # forward trace
    for k in range(n - 1):
        bubble_max_row(m, k)
        for i in range(k + 1, n):
            div = m[i][k] / m[k][k]
            m[i][-1] -= div * m[k][-1]
            for j in range(k, n):
                m[i][j] -= div * m[k][j]

    # check modified system for nonsingularity
    if is_singular(m):
        print('The system has infinite number of answers...')
        return

    # backward trace
    x = [0 for i in range(n)]
    for k in range(n - 1, -1, -1):
        x[k] = (m[k][-1] - sum([m[k][j] * x[j] for j in range(k + 1, n)])) / m[k][k]

    # Display results
    print(x)


def is_singular(m):
    """Check matrix for nonsingularity.
    :param m: matrix (list of lists)
    :return: True if system is nonsingular
    """
    for i in range(len(m)):
        if not m[i][i]:
            return True
    return False

solve_gauss(matrix)

[0.16810434673620603, -0.03511502217003547, 0.0903210276760497]


### Second task
*Matrix:*
$$\begin{pmatrix} 7.35272 & 0.88255 & -2.270052 \\ 0.88255 & 5.58351 & 0.528167 \\ -2.27005 & 0.528167 & 4.430329 \end{pmatrix} = \begin{pmatrix} 1 \\ 0 \\ 0 \end{pmatrix}$$ 

Решение методом Гаусса

In [14]:
solve_gauss(matrix)

[0.16810434673620603, -0.03511502217003547, 0.0903210276760497]


Преобразование системы к виду $x = Hx + g$ 


In [15]:
n = len(A)
E_matrix = np.eye(n)
D_matrix = np.zeros((n, n))

for i in range(n):
    D_matrix[i][i] = A[i][i]
    
# матрица обратная к D
D_inv = LA.inv(D_matrix)        
H = E_matrix - D_inv.dot(A)
g = D_inv.dot(B)
     

print("Матрица H:",'\n', H)
print("Столбец g:",'\n',g)

Матрица H: 
 [[ 0.         -0.12003041  0.30873636]
 [-0.15806366  0.         -0.09459408]
 [ 0.51238858 -0.1192162   0.        ]]
Столбец g: 
 [[0.13600409]
 [0.        ]
 [0.        ]]


 $||H||_\infty$

In [16]:
H_norm = LA.norm(H) 
H_norm

0.6483919812719281

Оценка погрешности  $||x^k-x^*||_\infty \lt \epsilon$, $\epsilon = 0.00001$

Из априорной оценки следует, что для достижения требуемой точности
необходимо выполнить количество итераций k, которое является
ближайшим натуральным числом, превышающим:
$k \ge \frac{\ln(\frac{\epsilon\cdot(1 - ||H||)}{||g||})}{\ln(||H||)}$

In [17]:
import math
# точность вычислений
eps = 10**(-3)

# норма столбца g_D
g_norm = max(np.abs(g))

# вычисляем правую часть неравенства
val_1 = np.log(eps*(1-H_norm)/g_norm)
val_2 = np.log(H_norm)
val_est =val_1/val_2

k_apr = math.ceil(val_est)
print("число итераций k: ", k_apr)

число итераций k:  14


Метод простейших итераций

In [21]:
import copy
xx = LA.solve(A, B)

x_k = np.zeros(n)
x_k_1 = g
k = 1
while np.max(np.abs(x_k_1 - x_k)) > eps and k != k_apr :
    x_k = copy.copy(x_k_1)
    vall = H.dot(x_k_1)
    x_k_1 = vall + g
    k += 1

print("Приближенное решение: ", x_k_1)
print("Точное решение: ", xx)
print("Фактическая погрешность: ", np.abs(x_k - x_k_1))

Приближенное решение:  [[ 0.16767497]
 [-0.03480106]
 [ 0.08957659]]
Точное решение:  [[ 0.16810435]
 [-0.03511502]
 [ 0.09032103]]
Фактическая погрешность:  [[0.00087405]
 [0.00032836]
 [0.0003966 ]]


In [23]:
apostr_est = H_norm/(1 - H_norm)*max(np.abs(x_k_1-x_k))     # апостериорная оценка
print("Апостериорная оценка: ", apostr_est)

apr_est = (H_norm**k)/(1 - H_norm)*g_norm       # априорная оценка
print("Априорная оценка", apr_est)

# уточним по Люстернику
w, v = LA.eig(H)      # собственные числа и векторы матрицы H
rho_H = max(abs(w))     #спектральный радиус матрицы H

x_Lust = x_k + 1/(1 - rho_H)*(x_k_1-x_k)
print("Уточненное решение по Люстернику: ", x_Lust)

print("Фактическая погрешность по Люстернику: ", np.abs(x_k - x_Lust))

Апостериорная оценка:  [0.00161181]
Априорная оценка [0.01863617]
Уточненное решение по Люстернику:  [[ 0.16842611]
 [-0.03508325]
 [ 0.08991743]]
Фактическая погрешность по Люстернику:  [[0.00162519]
 [0.00061054]
 [0.00073744]]


Метод Зейделя

In [25]:
H_L = np.zeros((n, n))
H_R = np.zeros((n, n))

for i in range(1, n):
    j = 0
    while j < i:
        H_L[i][j] = H[i][j]
        j += 1

for i in range(0, n):
    j = n-1
    while j >= i:
        H_R[i][j] = H[i][j]
        j -= 1

m_1 = LA.inv(E_matrix - H_L)
m_2 = m_1.dot(H_R)

x_k = np.zeros(n)
x_k_1 = m_1.dot(g)
k = 1
while (np.max(LA.norm(x_k_1-x_k))) > eps:
    x_k = copy.copy(x_k_1)
    x_k_1 = m_2.dot(x_k) + m_1.dot(g)
    k += 1



print("Решение, полученное методом Зейделя: ", x_k_1)
print("Фактическая погрешность: ", np.abs(x_k - x_k_1))
print("Потребовавшееся количество операций: ", k)

Решение, полученное методом Зейделя:  [[ 0.16802702]
 [-0.03508424]
 [ 0.09027774]]
Фактическая погрешность:  [[0.00027315]
 [0.00010872]
 [0.00015292]]
Потребовавшееся количество операций:  5


Метод верхней релаксации

In [27]:
q = 2/(1 + math.sqrt(1 - rho_H**2))         
x_k = np.zeros(n)
part_val_1 = LA.inv(E_matrix - q*H_L)
x_k_1 = part_val_1.dot(q*g)
k = 1

while np.max(np.abs(x_k_1 - x_k)) > eps and k!=28 :
    x_k = copy.copy(x_k_1)
    part_val = H_R.dot(x_k) - x_k + g
    x_k_1 = part_val_1.dot(x_k + q*part_val)
    k += 1
print("Решение методом верхней релаксации: ", x_k_1)
print("Число итераций", k)
print("Фактическая погрешность: ", np.abs(x_k_1 - x_k))

Решение методом верхней релаксации:  [[ 0.16807288]
 [-0.03510644]
 [ 0.09030904]]
Число итераций 4
Фактическая погрешность:  [[2.55454365e-04]
 [1.08706604e-04]
 [9.11780305e-05]]
